<a href="https://colab.research.google.com/github/Richish/deep_learning_with_python/blob/master/ch64_1dconv_for_sequence_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

# imdb classification using 1d conv

## Preparing data

In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_len=500
max_features=10_000

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features, maxlen=max_len)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train=sequence.pad_sequences(x_train, maxlen=max_len)
x_test=sequence.pad_sequences(x_test, maxlen=max_len)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

17465344/17464789 [==============================] - 2s 0us/step
25000 train sequences
20947 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 500)
x_test shape: (20947, 500)


## build, train, evaluate

1D convnets are structured in the same way as their 2D counterparts, which you used
in chapter 5: they consist of a stack of Conv1D and MaxPooling1D layers, ending in
either a global pooling layer or a Flatten layer, that turn the 3D outputs into 2D outputs,
allowing you to add one or more Dense layers to the model for classification or
regression.
One difference, though, is the fact that you can afford to use larger convolution
windows with 1D convnets. With a 2D convolution layer, a 3 × 3 convolution window
contains 3 × 3 = 9 feature vectors; but with a 1D convolution layer, a convolution window
of size 3 contains only 3 feature vectors. You can thus easily afford 1D convolution
windows of size 7 or 9.

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from keras.optimizers import RMSprop

model=Sequential()
model.add(Embedding(max_features, 128, input_length=max_len))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 128)          1280000   
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 1,315,937
Trainable params: 1,315,937
Non-trainable params: 0
____________________________________________

In [ ]:
model.compile(optimizer=RMSprop(lr=1e-4),loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train, y_train,epochs=10,batch_size=128,validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 70s 4ms/step - loss: 0.8688 - acc: 0.5159 - val_loss: 0.6929 - val_acc: 0.4932
Epoch 2/10
20000/20000 [==============================] - 70s 3ms/step - loss: 0.6758 - acc: 0.6179 - val_loss: 0.6744 - val_acc: 0.6374
Epoch 3/10
20000/20000 [==============================] - 70s 3ms/step - loss: 0.6442 - acc: 0.7255 - val_loss: 0.6382 - val_acc: 0.7166
Epoch 4/10
20000/20000 [==============================] - 70s 3ms/step - loss: 0.5744 - acc: 0.7843 - val_loss: 0.5421 - val_acc: 0.7734
Epoch 5/10
20000/20000 [==============================] - 70s 3ms/step - loss: 0.4456 - acc: 0.8342 - val_loss: 0.4255 - val_acc: 0.8196
Epoch 6/10
20000/20000 [==============================] - 69s 3ms/step - loss: 0.3645 - acc: 0.8591 - val_loss: 0.3998 - val_acc: 0.8294
Epoch 7/10
20000/20000 [==============================] - 69s 3ms/step - loss: 0.3160 - acc: 0.8606 - val_loss: 0.4106 - val_acc:

# Training and evaluating a simple 1D convnet on the Jena data

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)


13574144/13568290 [==============================] - 0s 0us/step


In [ ]:
df=pd.read_csv(csv_path)
df=df.drop(columns='Date Time')
df.keys()
mean=df.loc[:200_000].mean()
std=df.loc[:200_000].std()
df-=mean
df/=std
df.head()

float_data=df.to_numpy()
float_data.shape


(420551, 14)

In [ ]:
def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=6):
    """
    data: np.array > of full data set.
    lookback: no. of datapoints to look back in each sample.
    delay: no. of datapoints after current sample for which prediction is to be made, or which corresponds to y
    min_index: consider datapoints starting from this index only
    max_index: Take datapoint upto max this index only
    shuffle: shuffle the data before taking a batch out of data and yielding that
    batch_size: no. of samples to be yielded in a single batch
    step: not all data points will be considered, only datapoints at frequency- 'step' will be considered. 
            since data points aare at 10 min interval so step=6 implies only hourly data will be considered.
    return: a tuple of (samples, data_samples_at_delay). Samples is a 2d np array of shape: (batch_size, lookback, data.shape[-1]). Basically a a batch of samples, 
            where each sample contains a np array of shape(lookback, data.shape[-1])
            data_samples_at_delay is a 2d np array of shape: (batch_size, 1)- 1 since we only need temperature as output
    """
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback # which rows that will contain the end of sample data, delay will be relative to this row.
    while True:
        if shuffle:
            # rows will be equal to batch_size
            rows=np.random.randint(i, max_index, size=batch_size) # basically row indexes.
        else: # not shuffled
            if i+batch_size>max_index: # if reached end, need to rotate and go to beginning to get new batch
                i=min_index+lookback
            rows=np.arange(i, min(i+batch_size,max_index))
            i+=len(rows) # move pointer to where end of next batch would be
        # initializing samples and targets to 0s.
        samples=np.zeros((len(rows), lookback//step, data.shape[-1]))
        targets=np.zeros((len(rows),))
        # setting the value of samples and targets for each sample/target in a batch of batch_size
        for j, row in enumerate(rows):
            indeces=np.arange(row-lookback, row, step)
            samples[j]=data[indeces]
            targets[j]=data[row+delay][1] # we only need the temperature as output/label
        yield samples, targets


In [ ]:
lookback = 24*5*6 # 5 days data
step = 6
delay = 24*6 # 1 day's delay/prediction
batch_size = 128 # training batch size

train_gen = generator(float_data,
                lookback=lookback,
                delay=delay,
                min_index=0,
                max_index=200_000,
                shuffle=True,
                step=step,
                batch_size=batch_size)

val_gen = generator(float_data,
                lookback=lookback,
                delay=delay,
                min_index=200_001,
                max_index=300_000,
                shuffle=True,
                step=step,
                batch_size=batch_size)

test_gen = generator(float_data,
                lookback=lookback,
                delay=delay,
                min_index=300_001,
                max_index=None,
                shuffle=True,
                step=step,
                batch_size=batch_size)

val_steps = (300000 - 200001 - lookback) # how many val steps to look at whole data
test_steps = (len(float_data) - 300001 - lookback) # how many test steps to look at whole data

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',
input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [ ]:
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
steps_per_epoch=500,
epochs=20,
validation_data=val_gen,
validation_steps=val_steps/100)

Epoch 1/20
500/500 [==============================] - 6s 12ms/step - loss: 0.2187 - val_loss: 0.3595
Epoch 2/20
500/500 [==============================] - 6s 12ms/step - loss: 0.2149 - val_loss: 0.3834
Epoch 3/20
500/500 [==============================] - 6s 12ms/step - loss: 0.2110 - val_loss: 0.4002
Epoch 4/20
500/500 [==============================] - 6s 12ms/step - loss: 0.2088 - val_loss: 0.3910
Epoch 5/20
500/500 [==============================] - 6s 12ms/step - loss: 0.2058 - val_loss: 0.3824
Epoch 6/20
500/500 [==============================] - 6s 12ms/step - loss: 0.2025 - val_loss: 0.3858
Epoch 7/20
500/500 [==============================] - 6s 12ms/step - loss: 0.2007 - val_loss: 0.4427
Epoch 8/20
500/500 [==============================] - 6s 12ms/step - loss: 0.1991 - val_loss: 0.3648
Epoch 9/20
500/500 [==============================] - 6s 12ms/step - loss: 0.1985 - val_loss: 0.3951
Epoch 10/20
500/500 [==============================] - 6s 12ms/step - loss: 0.1940 - val_lo